# GLM-4 Evaluating Summarization with G-Eval Method

**This tutorial is available in English and is attached below the Chinese explanation**

本脚本参考了[OpenAI CookBook](https://github.com/openai/openai-cookbook/blob/main/examples/evaluation/How_to_eval_abstractive_summarization.ipynb)的代码，并使用 GLM 系列模型实现同样的功能。

This script refers to the code in [OpenAI CookBook](hhttps://github.com/openai/openai-cookbook/blob/main/examples/evaluation/How_to_eval_abstractive_summarization.ipynb) and implements the same functionality using the GLM family of models.

这个演示用G-Eval方法让用户可以使用自己的评判标准来评估和比较两个摘要，以找出并选择更有效的摘要以便使用。G-Eval方法通过两个步骤，首先根据给定的标准使用思维链 (CoT) 生成一系列评估步骤，然后给出得分。

This demo allows user to use their manual metrics to evaluate and compare two summaries to find out the more effective one. The G-Eval method works in two steps, using evaluation steps based on given criteria using Chain of Thought (CoT), then giving a score.


首先，设置环境变量并初始化 ZhipuAI client。

First, set up an environmental variable and initialize the ZhipuAI client.

In [1]:
import os
from zhipuai import ZhipuAI

os.environ["ZHIPUAI_API_KEY"] = "94a42f35989f725ac5e36d4a5b7b1469.SAxLFXQtLUYKgm2U"

client = ZhipuAI()

导入其他所有库，包括用于数据处理的 pandas 和用于显示结果的 tkinter。

Import all other libraries, including pandas for data manipulation and tkinter for displaying results.

In [2]:
import re
import pandas as pd
import tkinter as tk
from tkinter import ttk

现在，准备好我们要总结的文本和两份候选的摘要，以便进行评估。在这里，我准备了英国马洛小镇的介绍。文本和摘要的长度可长可短，我的文本为150字左右，摘要为50字左右。

Now, prepare the excerpt that we will be summarizing and two candidate summaries to evaluate. I have prepared an excerpt and two summaries about the coastal town Marlow located in England. While the length of these texts can vary, my excerpt is around 150 words and summaries are around 50 words.

In [3]:
excerpt = "The coastal town of Marlow was known for its pristine beaches, charming cottages, and vibrant local culture. Each summer, tourists flocked to its shores, eager to experience the town’s famous seafood festivals and lively street markets. Local artisans showcased their crafts, while fishermen brought in the freshest catches of the day. One of the town's most cherished traditions was the annual Lantern Festival. Every August, residents and visitors gathered to release hundreds of lanterns into the night sky, creating a breathtaking display of lights over the ocean. The festival symbolized hope and community, bringing people together in celebration. The town's history was rich with stories of seafarers and explorers, adding to its allure. As the sun set each evening, Marlow transformed into a picturesque scene, with golden hues reflecting off the water and the distant sound of waves lapping against the shore, making it a magical destination for all who visited."
eval_summary_1 = "Marlow, a coastal town, attracts tourists with its beaches, seafood festivals, and vibrant local culture. Its annual Lantern Festival in August, where lanterns light up the night sky, symbolizes hope and community. The town's history of seafarers adds allure, making Marlow a picturesque and magical destination."
eval_summary_2 = "Marlow's pristine beaches, seafood festivals, and annual Lantern Festival draw many visitors. The festival, held every August, brings the community together with a beautiful display of lanterns over the ocean. Rich in seafaring history, Marlow offers a magical, picturesque setting, especially at sunset."

准备基于G-Eval的评价提示模板，包括标准、步骤、文件、摘要和指标名称的占位符。

Prepare the template for the evaluation prompt, including placeholders for criteria, steps, document, summary, and metric name.


In [4]:
EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written for an article. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully. 
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""

给出指标的细则，每个度量都有自己的评判标准和步骤：这里原始的度量有相关性、连贯性、一致性、流畅性四项。

Give the details of the metrics. Each metric has its own rubric and steps: the original metrics here are relevance, coherence, consistency, and fluency.

In [5]:

# Metric 1: Relevance
RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""

RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""

# Metric 2: Coherence
COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""

COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""

# Metric 3: Consistency
CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""

CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""

# Metric 4: Fluency
FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""

FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""

定义 get_geval_score 函数，该函数与 ZhipuAI API 交互以提取分数。添加了多次重试，以提高从响应中获取数值分数的成功率。

Define the get_geval_score function that interacts with the ZhipuAI API to extract the score. Multiple retries is added to increase the success rate of obtaining a numerical score from the response.

In [6]:
def get_geval_score(criteria, steps, document, summary, metric_name, retries=5):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    for attempt in range(retries):
        response = client.chat.completions.create(
            model="glm-4-9b",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=100,
            top_p=1
        )
        response_text = response.choices[0].message.content.strip()

        score_match = re.search(r'\d+', response_text)
        if score_match:
            score_num = int(score_match.group())
            return score_num, response_text
        else:
            print(f"Warning: No valid score found in the response: {response_text}")

整合评判标准和需要比较的概括。

Collect the evaluation metrics and the summaries.

In [7]:
evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS),
}

summaries = {
    "Summary 1": eval_summary_1,
    "Summary 2": eval_summary_2
}

调用get_geval_score来计算每个概括在每个评判度量中的得分，并存到data中。

Call the get_geval_score function to calculate the scores of each summary on each metric, and save it to data.

In [8]:
data = {"Evaluation Type": [], "Summary Type": [], "Score": []}

for eval_type, (criteria, steps) in evaluation_metrics.items():
    for summ_type, summary in summaries.items():
        result = get_geval_score(criteria, steps, excerpt, summary, eval_type)
        if result is not None:
            data["Evaluation Type"].append(eval_type)
            data["Summary Type"].append(summ_type)
            data["Score"].append(result[0])

KeyboardInterrupt: 

最后，创建 DataFrame 来保存结果并使用 Tkinter 设置 GUI，以树状视图显示评估结果。

Lastly, create a DataFrame to store the results and set up the GUI using Tkinter to display the evaluation results in a treeview.

In [ ]:
pivot_df = pd.DataFrame(data).pivot(
    index="Evaluation Type", columns="Summary Type", values="Score"
)

root = tk.Tk()
root.title("Summary Evaluation Results")

tree = ttk.Treeview(root, columns=("Evaluation Type", "Summary 1", "Summary 2"), show='headings')
tree.heading("Evaluation Type", text="Evaluation Type")
tree.heading("Summary 1", text="Summary 1")
tree.heading("Summary 2", text="Summary 2")
tree.pack(fill=tk.BOTH, expand=True)

for index, row in pivot_df.iterrows():
    tree.insert("", "end", values=(index, row["Summary 1"], row["Summary 2"]))

root.mainloop()

评估以上的文本和概括得到以下表格。我们可以看出，根据我们给出的评估标准，第一个概括比第二个略好一些。

Evaluating the above texts and summaries yield the following table. We can see that the first summary is slightly better than the second according to the evaluation criteria we have given.

<img src="data/summary_evaluation.png" width="500" >